# CABEÇALHO

**Descrição:** este algoritmo foi elaborado para auxílio no projeto de pesquisa dos alunos Luiz Fonseca e Gabriela Ricardino, para o ano de 2023, do curso de Eng. de Energias Renováveis - UFPB, sob orientação dos professores Monica Carvalho e Raphael Abrahão.

**Objetivo:** auxílio no tratamento de dados meteorológicos obtidos através dos sites: 
  - [BDMEP](https://bdmep.inmet.gov.br/)
  - [NASA](https://power.larc.nasa.gov/data-access-viewer/)


**Desenvolvedor:** Luiz Fonseca [luiz.fonseca@cear.ufpb.br]

**Ano de desenvolvimento:** 2023

# IMPORTANTE

**Instruções, em vídeo, para uso do algoritmo:** https://youtu.be/M9Vh9MmBEmA
___

**Instruções para uso do algoritmo:**
1. Os arquivos **DEVEM** ser com extensão .csv;
2. Importe o arquivo no ícone de 'pasta' do lado esquerdo do Google Colab (quarto ícone de cima para baixo);
3. Para dados do INMET, são aceitos: dados diários ou dados mensais. Para dados da NASA: dados diários;
4. O tipo de pontuação para separação de decimais, para dados do INMET, **DEVE** ser PONTO;
5. Substitua, no algoritmo abaixo, os valores das variáveis entre as aspas simples:
  - variável *fonte*: os valores possíveis são **nasa** ou **inmet**, dentro das aspas simples;
  - variável *tipo_de_dado*: os valores possíveis são **diario** ou **mensal**, dentro das aspas simples;
  - variável *local_arquivo*: **path** do arquivo o qual você deseja trabalhar. Na pasta à esquerda, onde foi feito o upload do arquivo, basta clicar com o botão direito sobre ele e cliclar em 'Copiar caminho'. Substitua o que foi copiado entre as aspas simples.
6. Execute o algoritmo clicando no símbolo Screenshot_2.png no canto superior esquerdo do bloco;
7. O arquivo processado estará disponível na mesma pasta onde foi feito o upload do arquivo original, à esquerda. O nome do arquivo processado aparecerá como: 'Arquivo_Processado_[nome-do-arquivo-original].csv'.


> **OBS:** O algoritmo faz avaliação simultânea de todas as variáveis contidas no mesmo arquivo. Caso você queira avaliar as variáveis individualmente, separe-as em arquivos individuais, faça upload e execute o algoritmo para cada arquivo, individualmente.



# ALGORITMO

In [11]:
                    # Abaixo, inserir sempre dentro das aspas simples

fonte =             'inmet'  # possíveis opções: 'nasa' ou 'inmet'

tipo_de_dado =      'mensal'   # possíveis opções: 'diario' ou 'mensal'

local_arquivo =     'caminho'


'''

      NÃO ALTERAR, OU INSERIR OU EXCLUIR ABSOLUTAMENTE NADA DO QUE VIER ABAIXO DESTA LINHA DO CÓDIGO !!!

'''




import pandas as pd
import numpy as np

def dados_diarios_inmet():
  df = pd.read_csv(local_arquivo, sep=';', skiprows=10, decimal='.') 
                  # lê o arquivo e pula o cabeçalho. Se o INMET alterar o cabeçalho futuramente, deve-se
                  # alterar o número do skiprows. Além disso, também deverá ser alterada a leitura feita na variável 'df_cabecalho'.

  df['Data Medicao'] = pd.to_datetime(df['Data Medicao'],dayfirst=True) # transforma os valores da coluna Data Medicao em valores de datatime

  num_colunas = df.shape[1] # retorna a quantidade total de colunas existentes na tabela. Esse valor pode sofrer alterações no futuro, caso o INMET decida
                            # configurar o export de dados e retirar a coluna vazia (sempre última coluna) criada nas planilhas. Caso isso aconteça,
                            # o código deverá ser alterado para se adequar a coluna faltante.

  df_temp = pd.DataFrame({df.columns[-1]:['STOP']}) # cria o valor STOP para finalizar a leitura do DataFrame
  df = df.append(df_temp, ignore_index=True) # adiciona o valor STOP ao final da planilha original

  i = 0
  lista_indices = [] # lista onde ficarão armazenados os índices de linhas que contêm valores Null
  while(str(df.iat[i,-1]) != 'STOP'): # comando de repetição para seleção das linhas cujos valores são 'Null' (ou NaN)
    for k in range(1,num_colunas-1):
      if (str(df.iat[i,k]) == 'nan'):
        lista_indices.append(i)
        break;
      else:
        continue;
    i += 1
    if (str(df.iat[i,-1]) == 'STOP'):
      break;

  df.drop(df.index[lista_indices], inplace=True) # comando para exclusão das linhas cujos valores são 'Null' (ou NaN)

  df.reset_index() # reseta o index padrão da planilha para ser usado com o comando .iloc()
  df['Meses'] = df['Data Medicao'].dt.month # cria temporariamente a coluna 'Meses'
  df['Anos'] = df['Data Medicao'].dt.year # cria temporariamente a coluna 'Anos'

  lista_indices_2 = [] # lista (de listas) onde ficará armazenado o 'ano + mês' que estão incompletos
  df_qtd_dias_por_mes_e_ano = df.groupby(['Anos','Meses']).count() # DataFrame com a quantidade de dias de cada ano
  for i in range (0, len(df_qtd_dias_por_mes_e_ano)):
    ano = str(int(df_qtd_dias_por_mes_e_ano.index[i][0]))
    mes = str(int(df_qtd_dias_por_mes_e_ano.index[i][1]))
    qntd_correta_dias = pd.Period(ano + '-' + mes).days_in_month # quantidade correta de dias que deveria existir na data especificada
    if df_qtd_dias_por_mes_e_ano.iloc[i,0] != qntd_correta_dias:
      lista_indices_2.append(str(float(mes)) + '-' + str(float(ano)))

  lista_mes_e_ano = [] # lista com valores do Mês e Ano juntos
  for i in range (len(df)): # comando de repetição para juntar o Mês com o Ano
    lista_mes_e_ano.append(str(df.iloc[i,-2]) + '-' + str(df.iloc[i,-1]))
  df['Meses+Anos'] = lista_mes_e_ano # cria a coluna 'Meses+Anos' no DataFrame

  lista_indices_3 = [] # lista onde ficarão armazenados os índices das linhas que serão excluídas por não pertencer a um mês de dados completos
  for k in range(len(df)): # comando de repetição para seleção das linhas que serão excluídas por não pertencer a um mês de dados completos
    if (str(df.iat[k,-4]) == 'STOP'):
      break;
    if (df.iat[k,-1] in lista_indices_2):
      lista_indices_3.append(k)
    else:
      continue;

  df.drop(df.index[lista_indices_3], inplace=True) # comando para exclusão das linhas cujo mês não está completo

  df_media_por_mes_e_ano = df.groupby(['Meses','Anos'],sort=False).mean() # DataFrame com a media total de cada mês de cada ano (para cada variáveis)
  df_novo = pd.DataFrame (index=np.arange(0), columns=np.arange(12)) # cria um novo DataFrame vazio com 12 colunas
  df_cabecalho = pd.read_csv(local_arquivo, nrows=10, usecols=[0], sep=';', decimal='.') # adiciona o cabeçalho do arquivo original ao novo DataFrame
  df_novo = pd.concat([df_cabecalho,df_novo], sort=False) # concatena o cabeçalho ao novo dataframe

  nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
  x = 1 # valor inicial da PRIMEIRA COLUNA que contém variável para ser analisada
  h = 0 # valor que diz respeito à coluna (variável) que está sendo analisada
  condicao_inicial = True # condição inicial para que os cdigos a seguir sejam inicializados

  while (condicao_inicial):
    nova_linha_cabecalho = [df.columns.values[x]] # altera o 'x' para correr entre as colunas, começando a partir do valor 1 (onde estará a primeira variável)
    
    if ('Unnamed: ' + str(x)) in nova_linha_cabecalho: # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
      break;                                           # Este comando é igual ao comando que está no final deste while principal. Este comando desta linha é apenas por precaução
    
    else:
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False) # concateca o DataFrame novo com uma linha de identifica a primeira variável
                                                                                    # esse código se repete para todas as variáveis analisadas

      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False) # concatena o dataframe novo com o header contendo o nome dos meses (Ano + Jan-Dez)
                                                                                  # esse código se repetirá para todas as variáveis analisadas

      f = 0 # valor que diz respeito à linha atual do ano que está sendo analisado
      b = 0 # valor que diz respeito à linha atual que está sendo analisada
      u = 0 # valor referente ao primeiro dado que será adicionado ao DataFrame final, na ordem correta
      p = 0 # valor de incremento das linhas do DataFrame (de zero até a última linha)
      cond = True # condição para início do comando while

      planilha_final = [] # planilha final que agrupa os dados organizados corretamente em meses e anos

      while(cond):
        ano_temp = df_media_por_mes_e_ano.index[f][1] # guarda, temporariamente, o valor do ano que está sendo analisado naquele momento
        mes_para_linha = [] # lista temporária com os valores dos meses
        mes_para_planilha = [] # lista temporária com os valores dos meses na ordem correta (de jan a dez) 

        while(df_media_por_mes_e_ano.index[b][1] == ano_temp): # comando que salva os valores referentes ao meses do ano que está sendo analisado
          mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h]) # adiciona os valores dos meses à planilha temporária
          b += 1
          if (len(df_media_por_mes_e_ano) == b): # finaliza o comando de repetição caso chegue na última linha do DataFrame e, consequentemente, não há mais dados para analisar
            break; 
        
        for i in range (0,13): # comando FOR que organiza os meses de forma correta (de Jan a Fev. Caso não exista o dado de determinado mês, será atribuído '-' àquele mês
          if p >= len(df_media_por_mes_e_ano): 
            mes_para_planilha.append('-')
            p += 1
            continue;
          if i == 0:
            mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
            continue;
          if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
            if (len(mes_para_linha) == u):
              continue;
            else:
              mes_para_planilha.append(mes_para_linha[u])
              u += 1
              p += 1
          else:
            mes_para_planilha.append('-')
            continue;         # --- fim do comando FOR
        u = 0
        
        planilha_final.append(mes_para_planilha) # comando que adiciona os meses em ordem correta, em uma única planilha, para cada variável que existir na planilha original
        f = b
        
        if (len(df_media_por_mes_e_ano) == f): # comando extra para finalizar a repetição quando o DataFrame chegue na última linha
          cond = False
          break;

      df_para_adicionar = pd.DataFrame(planilha_final) # transforma a planilha final, com os meses em ordem correta, em um DataFrame

      df_novo = df_novo.append(df_para_adicionar, ignore_index=True) # adiciona o DataFrame com os valores organizados dos meses ao DataFrame novo

      pula_linha = ['*','*','*'] # 'linha' aleatória para divir os dados das variáveis
      df_pula_linha = pd.DataFrame(pula_linha)

      df_novo = df_novo.append(df_pula_linha, ignore_index=True) # adiciona uma linha aleatória de '*' para dividir (separar) as variáveis da planilha

      h += 1 # aumenta o valor de 'h' para percorrer a próxima coluna (para análise dos valores nesta coluna)
      x += 1 # aumenta o valor de 'x' para percorrer a próxima coluna (para análise do nome da coluna)

      if ('Unnamed: ' + str(x)) in nova_linha_cabecalho: # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
        condicao_inicial = False # comando para garantir que o while principal não será executado novamente
        break;

  df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep=';') # comando final que cria a nova planilha no excel (formato .CSV) para ser baixada
  return print('Fim do processamento! Arquivo disponível para download ao lado   <<<<----')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def dados_mensais_inmet():
  df = pd.read_csv(local_arquivo, sep=';', skiprows=10, decimal='.') 
                  # lê o arquivo e pula o cabeçalho. Se o INMET alterar o cabeçalho futuramente, deve-se
                  # alterar o número do skiprows. Além disso, também deverá ser alterada a leitura feita na variável 'df_cabecalho'.

  df['Data Medicao'] = pd.to_datetime(df['Data Medicao'],dayfirst=True) # transforma os valores da coluna Data Medicao em valores de datatime

  num_colunas = df.shape[1] # retorna a quantidade total de colunas existentes na tabela. Esse valor pode sofrer alterações no futuro, caso o INMET decida
                            # configurar o export de dados e retirar a coluna vazia (sempre última coluna) criada nas planilhas. Caso isso aconteça,
                            # o código deverá ser alterado para se adequar a coluna faltante.

  df_temp = pd.DataFrame({df.columns[-1]:['STOP']}) # cria o valor STOP para finalizar a leitura do DataFrame
  df = df.append(df_temp, ignore_index=True) # adiciona o valor STOP ao final da planilha original

  i = 0
  lista_indices = [] # lista onde ficarão armazenados os índices de linhas que contêm valores Null
  while(str(df.iat[i,-1]) != 'STOP'): # comando de repetição para seleção das linhas cujos valores são 'Null' (ou NaN)
    for k in range(1,num_colunas-1):
      if (str(df.iat[i,k]) == 'nan'):
        lista_indices.append(i)
        break;
      else:
        continue;
    i += 1
    if (str(df.iat[i,-1]) == 'STOP'):
      break;

  df.drop(df.index[lista_indices], inplace=True) # comando para exclusão das linhas cujos valores são 'Null' (ou NaN)

  df.reset_index() # reseta o index padrão da planilha para ser usado com o comando .iloc()
  df['Meses'] = df['Data Medicao'].dt.month # cria temporariamente a coluna 'Meses'
  df['Anos'] = df['Data Medicao'].dt.year # cria temporariamente a coluna 'Anos'

  lista_mes_e_ano = [] # lista com valores do Mês e Ano juntos
  for i in range (len(df)): # comando de repetição para juntar o Mês com o Ano
    lista_mes_e_ano.append(str(df.iloc[i,-2]) + '-' + str(df.iloc[i,-1]))
  df['Meses+Anos'] = lista_mes_e_ano # cria a coluna 'Meses+Anos' no DataFrame

  for i in range (len(df)): # comando de repetição para ler as linhas e passar os valores de str para float (e trocar ',' para '.')     ***** NOVAS LINHAS, em comparação ao programa de análise de dados diários
    for k in range(1,num_colunas-1):
      string_temp = str(df.iat[i,k])
      float_temp = string_temp.replace(',','.')
      df.iat[i,k] = float(float_temp)

  df_media_por_mes_e_ano = df.groupby(['Meses','Anos'],sort=False).mean() # DataFrame com a media total de cada mês de cada ano (para cada variáveis)
  df_novo = pd.DataFrame (index=np.arange(0), columns=np.arange(12)) # cria um novo DataFrame vazio com 12 colunas
  df_cabecalho = pd.read_csv(local_arquivo, nrows=10, usecols=[0], sep=';', decimal='.') # adiciona o cabeçalho do arquivo original ao novo DataFrame
  df_novo = pd.concat([df_cabecalho,df_novo], sort=False) # concatena o cabeçalho ao novo dataframe

  nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
  x = 1 # valor inicial da PRIMEIRA COLUNA que contém variável para ser analisada
  h = 0 # valor que diz respeito à coluna (variável) que está sendo analisada          ********* Essa variável é diferente do programa de análise DIÁRIO, pois no diário o group by não lê a coluna de data, mas nos dados mensais ela lê **********
  condicao_inicial = True # condição inicial para que os cdigos a seguir sejam inicializados

  while (condicao_inicial):
    nova_linha_cabecalho = [df.columns.values[x]] # altera o 'x' para correr entre as colunas, começando a partir do valor 1 (onde estará a primeira variável)
    
    if ('Unnamed: ' + str(x)) in nova_linha_cabecalho: # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
      break;                                           # Este comando é igual ao comando que está no final deste while principal. Este comando desta linha é apenas por precaução
    
    else:
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False) # concateca o DataFrame novo com uma linha de identifica a primeira variável
                                                                                    # esse código se repete para todas as variáveis analisadas

      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False) # concatena o dataframe novo com o header contendo o nome dos meses (Ano + Jan-Dez)
                                                                                  # esse código se repetirá para todas as variáveis analisadas

      f = 0 # valor que diz respeito à linha atual do ano que está sendo analisado
      b = 0 # valor que diz respeito à linha atual que está sendo analisada
      u = 0 # valor referente ao primeiro dado que será adicionado ao DataFrame final, na ordem correta
      p = 0 # valor de incremento das linhas do DataFrame (de zero até a última linha)
      cond = True # condição para início do comando while

      planilha_final = [] # planilha final que agrupa os dados organizados corretamente em meses e anos

      while(cond):
        ano_temp = df_media_por_mes_e_ano.index[f][1] # guarda, temporariamente, o valor do ano que está sendo analisado naquele momento
        mes_para_linha = [] # lista temporária com os valores dos meses
        mes_para_planilha = [] # lista temporária com os valores dos meses na ordem correta (de jan a dez) 

        while(df_media_por_mes_e_ano.index[b][1] == ano_temp): # comando que salva os valores referentes ao meses do ano que está sendo analisado
          mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h]) # adiciona os valores dos meses à planilha temporária
          b += 1
          if (len(df_media_por_mes_e_ano) == b): # finaliza o comando de repetição caso chegue na última linha do DataFrame e, consequentemente, não há mais dados para analisar
            break; 
        
        for i in range (0,13): # comando FOR que organiza os meses de forma correta (de Jan a Fev. Caso não exista o dado de determinado mês, será atribuído '-' àquele mês
          if p >= len(df_media_por_mes_e_ano): 
            mes_para_planilha.append('-')
            p += 1
            continue;
          if i == 0:
            mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
            continue;
          if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
            if (len(mes_para_linha) == u):
              continue;
            else:
              mes_para_planilha.append(mes_para_linha[u])
              u += 1
              p += 1
          else:
            mes_para_planilha.append('-')
            continue;         # --- fim do comando FOR
        u = 0
        
        planilha_final.append(mes_para_planilha) # comando que adiciona os meses em ordem correta, em uma única planilha, para cada variável que existir na planilha original
        f = b
        
        if (len(df_media_por_mes_e_ano) == f): # comando extra para finalizar a repetição quando o DataFrame chegue na última linha
          cond = False
          break;

      df_para_adicionar = pd.DataFrame(planilha_final) # transforma a planilha final, com os meses em ordem correta, em um DataFrame

      df_novo = df_novo.append(df_para_adicionar, ignore_index=True) # adiciona o DataFrame com os valores organizados dos meses ao DataFrame novo

      pula_linha = ['*','*','*'] # 'linha' aleatória para divir os dados das variáveis
      df_pula_linha = pd.DataFrame(pula_linha)

      df_novo = df_novo.append(df_pula_linha, ignore_index=True) # adiciona uma linha aleatória de '*' para dividir (separar) as variáveis da planilha

      h += 1 # aumenta o valor de 'h' para percorrer a próxima coluna (para análise dos valores nesta coluna)
      x += 1 # aumenta o valor de 'x' para percorrer a próxima coluna (para análise do nome da coluna)

      if ('Unnamed: ' + str(x)) in nova_linha_cabecalho: # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
        condicao_inicial = False # comando para garantir que o while principal não será executado novamente
        break;

  df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep=';') # comando final que cria a nova planilha no excel (formato .CSV) para ser baixada
  return print('Fim do processamento! Arquivo disponível para download ao lado   <<<<----')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def dados_diarios_nasa():
  df = pd.read_csv(local_arquivo,sep=',',skiprows=9, decimal='.')
                  # lê o arquivo .csv e pula o cabeçalho. Se a NASA alterar o cabeçalho futuramente, deve-se
                  # alterar o número do skiprows. Além disso, também deverá ser alterada a leitura feita na variável 'df_cabecalho'. 

  df['STOP'] = 'STOP' # cria uma nova coluna 'STOP'.

  num_colunas = df.shape[1] # retorna a quantidade total de colunas existentes na tabela.

  i = 0
  f = 0 # condição de parada da função while abaixo
  lista_indices = [] # lista onde ficarão armazenados os índices de linhas que contêm valores considerados 'nulos'
  while(f < len(df)): # comando de repetição para seleção das linhas cujos valores são 'Null' (ou NaN)
    for k in range(3,num_colunas):
      if (df.iat[i,k] == -999.0):
        lista_indices.append(i)
        break;
      else:
        continue;
    i += 1
    f += 1
    if (f == len(df)):
      break;

  df.drop(df.index[lista_indices], inplace=True) # comando para exclusão das linhas cujos valores são considerados 'nulos' (ou -999.0)

  df.reset_index() # reseta o index padrão da planilha para ser usado com o comando .iloc()

  lista_indices_2 = [] # lista (de listas) onde ficará armazenado o 'ano + mês' que estão incompletos
  df_qtd_dias_por_mes_e_ano = df.groupby(['YEAR','MO']).count() # DataFrame com a quantidade de dias de cada ano
  for i in range (0, len(df_qtd_dias_por_mes_e_ano)):
    ano = str(int(df_qtd_dias_por_mes_e_ano.index[i][0]))
    mes = str(int(df_qtd_dias_por_mes_e_ano.index[i][1]))
    qntd_correta_dias = pd.Period(ano + '-' + mes).days_in_month # quantidade correta de dias que deveria existir na data especificada
    if df_qtd_dias_por_mes_e_ano.iloc[i,0] != qntd_correta_dias:
      lista_indices_2.append(str(int(ano)) + '-' + str(int(mes)))

  lista_mes_e_ano = [] # lista com valores do Mês e Ano juntos
  for i in range (len(df)): # comando de repetição para juntar o Mês com o Ano
    lista_mes_e_ano.append(str(int(df.iloc[i,0])) + '-' + str(int(df.iloc[i,1])))
  df['Meses+Anos'] = lista_mes_e_ano # cria a coluna 'Meses+Anos' no DataFrame

  lista_indices_3 = [] # lista onde ficarão armazenados os índices das linhas que serão excluídas por não pertencer a um mês de dados completos
  for k in range(len(df)): # comando de repetição para seleção das linhas que serão excluídas por não pertencer a um mês de dados completos
    if (df.iat[k,-1] in lista_indices_2):
      lista_indices_3.append(k)
    else:
      continue;

  df.drop(df.index[lista_indices_3], inplace=True) # comando para exclusão das linhas cujo mês não está completo

  df_media_por_mes_e_ano = df.groupby(['MO','YEAR'],sort=False).mean() # DataFrame com a media total de cada mês de cada ano (para cada variáveis)
  df_novo = pd.DataFrame (index=np.arange(0), columns=np.arange(12)) # cria um novo DataFrame vazio com 12 colunas
  df_cabecalho = pd.read_csv(local_arquivo, nrows=9, usecols=[0], sep=',', decimal='.') # adiciona o cabeçalho do arquivo original ao novo DataFrame
  df_novo = pd.concat([df_cabecalho,df_novo], sort=False) # concatena o cabeçalho ao novo dataframe

  nova_linha_header = ['Ano','Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
  x = 3 # valor inicial da PRIMEIRA COLUNA que contém variável para ser analisada
  h = 1 # valor que diz respeito à coluna (variável) que está sendo analisada
  condicao_inicial = True # condição inicial para que os cdigos a seguir sejam inicializados

  while (condicao_inicial):
    nova_linha_cabecalho = [df.columns.values[x]] # altera o 'x' para correr entre as colunas, começando a partir do valor 1 (onde estará a primeira variável)
    
    if ('STOP' in nova_linha_cabecalho): # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
      break;                             # Este comando é igual ao comando que está no final deste while principal. Este comando desta linha é apenas por precaução
    
    else:
      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_cabecalho)], sort=False) # concateca o DataFrame novo com uma linha de identifica a primeira variável
                                                                                    # esse código se repete para todas as variáveis analisadas

      df_novo = pd.concat([df_novo,pd.DataFrame(nova_linha_header).T], sort=False) # concatena o dataframe novo com o header contendo o nome dos meses (Ano + Jan-Dez)
                                                                                  # esse código se repetirá para todas as variáveis analisadas

      f = 0 # valor que diz respeito à linha atual do ano que está sendo analisado
      b = 0 # valor que diz respeito à linha atual que está sendo analisada
      u = 0 # valor referente ao primeiro dado que será adicionado ao DataFrame final, na ordem correta
      p = 0 # valor de incremento das linhas do DataFrame (de zero até a última linha)
      cond = True # condição para início do comando while

      planilha_final = [] # planilha final que agrupa os dados organizados corretamente em meses e anos

      while(cond):
        ano_temp = df_media_por_mes_e_ano.index[f][1] # guarda, temporariamente, o valor do ano que está sendo analisado naquele momento
        mes_para_linha = [] # lista temporária com os valores dos meses
        mes_para_planilha = [] # lista temporária com os valores dos meses na ordem correta (de jan a dez) 

        while(df_media_por_mes_e_ano.index[b][1] == ano_temp): # comando que salva os valores referentes ao meses do ano que está sendo analisado
          mes_para_linha.append(df_media_por_mes_e_ano.iloc[b,h]) # adiciona os valores dos meses à planilha temporária
          b += 1
          if (len(df_media_por_mes_e_ano) == b): # finaliza o comando de repetição caso chegue na última linha do DataFrame e, consequentemente, não há mais dados para analisar
            break; 
        
        for i in range (0,13): # comando que organiza os meses de forma correta (de Jan a Fev. Caso não exista o dado de determinado mês, será atribuído '-' àquele mês
          if p >= len(df_media_por_mes_e_ano): 
            mes_para_planilha.append('-')
            p += 1
            continue;
          if i == 0:
            mes_para_planilha.append(df_media_por_mes_e_ano.index[p][1])
            continue;
          if (float(df_media_por_mes_e_ano.index[p][0]) == float(i)):
            if (len(mes_para_linha) == u):
              continue;
            else:
              mes_para_planilha.append(mes_para_linha[u])
              u += 1
              p += 1
          else:
            mes_para_planilha.append('-')
            continue;         # --- fim do comando FOR
        u = 0
        
        planilha_final.append(mes_para_planilha) # comando que adiciona os meses em ordem correta, em uma única planilha, para cada variável que existir na planilha original
        f = b
        
        if (len(df_media_por_mes_e_ano) == f): # comando extra para finalizar a repetição quando o DataFrame chegue na última linha
          cond = False
          break;
      #------ fim while
      df_para_adicionar = pd.DataFrame(planilha_final) # transforma a planilha final, com os meses em ordem correta, em um DataFrame

      df_novo = df_novo.append(df_para_adicionar, ignore_index=True) # adiciona o DataFrame com os valores organizados dos meses ao DataFrame novo

      ultima_linha = ['*','*','*'] # 'linha' aleatória final
      df_ultima_linha = pd.DataFrame(ultima_linha)

      df_novo = df_novo.append(df_ultima_linha, ignore_index=True) # adiciona uma linha final
      
      h += 1 # aumenta o valor de 'h' para percorrer a próxima coluna (para análise dos valores nesta coluna)
      x += 1 # aumenta o valor de 'x' para percorrer a próxima coluna (para análise do nome da coluna)

      if ('STOP' in nova_linha_cabecalho): # comando que finaliza a leitura das colunas caso todas as variáveis já tenham sido analisadas
        condicao_inicial = False # comando para garantir que o while principal não será executado novamente
        break;

  df_novo.to_csv('Arquivo_Processado_' + str(local_arquivo[9:]), sep=';') # comando final que cria a nova planilha no excel (formato .CSV) para ser baixada
  return print('Fim do processamento! Arquivo disponível para download ao lado   <<<<----')

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------

fonte = fonte.lower().strip() # corrige a palavra digitada pelo usuário 
tipo_de_dado = tipo_de_dado.lower().strip().replace('á','a') # corrige a palavra digitada pelo usuário 

if fonte == 'nasa' and tipo_de_dado == 'diario':
  dados_diarios_nasa()
elif fonte == 'inmet' and tipo_de_dado == 'diario':
  dados_diarios_inmet()
elif fonte == 'inmet' and tipo_de_dado == 'mensal':
  dados_mensais_inmet()
else:
  print('ERRO! Verifique se você inseriu as informações de maneira correta nas variáveis: "fonte", "tipo_de_dado" e "local_arquivo".')

Fim do processamento! Arquivo disponível para download ao lado   <<<<----
